In [4]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.1.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:2 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:3 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:4 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:7 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:8 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:9 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [83.3 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:11 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ Packages [101 kB]
Hit:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:14 http://ppa.launc

In [5]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M16-Amazon-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [6]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Sports_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.csv(SparkFiles.get("amazon_reviews_us_Sports_v1_00.tsv.gz"), sep="\t", header=True)
df.show(50)

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   48945260|R1WBPB8MDCCN8F|B012P7UPSM|     409940130|Chicago Blackhawk...|          Sports|          5|            0|          0|   N|                N|   LOVE IT. 6 stars!|Bought this last ...| 2015-08-31|
|         US|    5782091|R32M0YEWV77XG8|B001GQ3VHG|     657746679|Copag Poker Size ...|          Sports|          5|    

In [7]:
from pyspark.sql.functions import to_date

In [8]:
# Create the vine_table. DataFrame
vine_table = df.select(["review_id", "star_rating", "helpful_votes", "total_votes","vine", "verified_purchase"])
vine_table.show(20)

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R1WBPB8MDCCN8F|          5|            0|          0|   N|                N|
|R32M0YEWV77XG8|          5|            1|          1|   N|                Y|
| RR8V7WR27NXJ5|          1|            0|          0|   N|                Y|
|R1MHO5V9Z932AY|          5|            0|          0|   N|                Y|
|R16PD71086BD2V|          5|            0|          1|   N|                N|
|R1Z8IFGWTRWXT6|          3|            0|          0|   N|                Y|
|R3AUMSHAW73HWN|          4|            2|          3|   N|                N|
|R2KWDWFOHGX6FL|          5|            0|          0|   N|                Y|
|R3H9543FWBWFBU|          5|            1|          1|   N|                Y|
| RUANXOQ9W3OU5|          5|            0|          0|   N|     

In [9]:
D2_df = vine_table.filter("total_votes>=20")
D2_df.show(10)

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R1B4753QMTD8ON|          5|           32|         32|   N|                N|
|R28BVHOEL6GCHZ|          5|           21|         21|   N|                Y|
|R27IB4BRKO71I0|          2|           16|         20|   N|                Y|
| RJ86OW12Y4NRL|          5|          124|        128|   N|                Y|
|R3KRABOWCH6ZE4|          1|           48|         65|   N|                Y|
|R1OAKARA30DIWX|          4|           83|         89|   N|                Y|
|R36LSV4WQ4VB1K|          5|           17|         22|   N|                N|
|R18BTOE5UGN7UA|          5|           26|         28|   N|                Y|
|R1DXLGDSK2WMLD|          4|           22|         22|   N|                N|
| RZG2OHQ12YYZI|          5|           30|         31|   N|     

In [10]:
from pyspark.sql.functions import col
D2_Percent_Helpful = D2_df.withColumn('percent_helpful_votes',D2_df['helpful_votes']/D2_df['total_votes']).alias('percent_helpful_votes').filter(col("percent_helpful_votes") >= 0.5)
D2_Percent_Helpful.show(10)

+--------------+-----------+-------------+-----------+----+-----------------+---------------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|percent_helpful_votes|
+--------------+-----------+-------------+-----------+----+-----------------+---------------------+
|R1B4753QMTD8ON|          5|           32|         32|   N|                N|                  1.0|
|R28BVHOEL6GCHZ|          5|           21|         21|   N|                Y|                  1.0|
|R27IB4BRKO71I0|          2|           16|         20|   N|                Y|                  0.8|
| RJ86OW12Y4NRL|          5|          124|        128|   N|                Y|              0.96875|
|R3KRABOWCH6ZE4|          1|           48|         65|   N|                Y|   0.7384615384615385|
|R1OAKARA30DIWX|          4|           83|         89|   N|                Y|   0.9325842696629213|
|R36LSV4WQ4VB1K|          5|           17|         22|   N|                N|   0.7727272727272727|


In [13]:
# Filter on exact Vines with Y
Vine_Review = D2_Percent_Helpful.filter(D2_Percent_Helpful["vine"] == "Y").show(10)

+--------------+-----------+-------------+-----------+----+-----------------+---------------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|percent_helpful_votes|
+--------------+-----------+-------------+-----------+----+-----------------+---------------------+
|R32OMS1LSDRCOB|          4|           27|         31|   Y|                N|   0.8709677419354839|
|R2IPKK4I0XRCFR|          4|           16|         21|   Y|                N|   0.7619047619047619|
| RXYYJ8SGI9IX6|          5|           36|         41|   Y|                N|   0.8780487804878049|
|R1SAUBFC7T6JHC|          5|           73|         75|   Y|                N|   0.9733333333333334|
|R329HVJKKHWSG3|          5|           59|         59|   Y|                N|                  1.0|
| RZL21K50M52KW|          5|           22|         22|   Y|                N|                  1.0|
|R24RC5J2D92SBE|          5|           19|         20|   Y|                N|                 0.95|


In [14]:
# Filter on the Vines with N
Non_Vine_Review = D2_Percent_Helpful.filter(D2_Percent_Helpful["vine"] == "N").show(10)

+--------------+-----------+-------------+-----------+----+-----------------+---------------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|percent_helpful_votes|
+--------------+-----------+-------------+-----------+----+-----------------+---------------------+
|R1B4753QMTD8ON|          5|           32|         32|   N|                N|                  1.0|
|R28BVHOEL6GCHZ|          5|           21|         21|   N|                Y|                  1.0|
|R27IB4BRKO71I0|          2|           16|         20|   N|                Y|                  0.8|
| RJ86OW12Y4NRL|          5|          124|        128|   N|                Y|              0.96875|
|R3KRABOWCH6ZE4|          1|           48|         65|   N|                Y|   0.7384615384615385|
|R1OAKARA30DIWX|          4|           83|         89|   N|                Y|   0.9325842696629213|
|R36LSV4WQ4VB1K|          5|           17|         22|   N|                N|   0.7727272727272727|


In [15]:
# Get total Count
Unique_Reviews = D2_Percent_Helpful.drop_duplicates(subset=['review_id']).count()
print(Unique_Reviews)

61948


In [16]:
# Y Vine Data 
Vine_Reviews = D2_Percent_Helpful.filter(D2_Percent_Helpful["vine"] == "Y").count()
Vine_Reviews_5Star = D2_Percent_Helpful.filter(D2_Percent_Helpful["vine"] == "Y").filter(D2_Percent_Helpful["star_rating"] == "5").count()
Percentage_5_Star = Vine_Reviews_5Star / Vine_Reviews

print(Vine_Reviews)
print(Vine_Reviews_5Star)
print(Percentage_5_Star)

334
139
0.4161676646706587


In [17]:
# N Vine Data 
Non_Vine_Reviews = D2_Percent_Helpful.filter(D2_Percent_Helpful["vine"] == "N").count()
Non_Vine_Reviews_5Star = D2_Percent_Helpful.filter(D2_Percent_Helpful["vine"] == "N").filter(D2_Percent_Helpful["star_rating"] == "5").count()
Non_Percentage_5_Star = Non_Vine_Reviews_5Star / Non_Vine_Reviews

print(Non_Vine_Reviews)
print(Non_Vine_Reviews_5Star)
print(Non_Percentage_5_Star)

61614
32665
0.5301554841432142


In [11]:
# Y Vine Data 
Vine_Reviews = D2_Percent_Helpful.filter(D2_Percent_Helpful["vine"] == "Y").count()
Vine_Reviews_5Star = D2_Percent_Helpful.filter(D2_Percent_Helpful["vine"] == "Y").filter(D2_Percent_Helpful["star_rating"] >= "3").count()
Percentage_5_Star = Vine_Reviews_5Star / Vine_Reviews

print(Vine_Reviews)
print(Vine_Reviews_5Star)
print(Percentage_5_Star)

334
313
0.937125748502994


In [12]:
# N Vine Data 
Non_Vine_Reviews = D2_Percent_Helpful.filter(D2_Percent_Helpful["vine"] == "N").count()
Non_Vine_Reviews_5Star = D2_Percent_Helpful.filter(D2_Percent_Helpful["vine"] == "N").filter(D2_Percent_Helpful["star_rating"] >= "3").count()
Non_Percentage_5_Star = Non_Vine_Reviews_5Star / Non_Vine_Reviews

print(Non_Vine_Reviews)
print(Non_Vine_Reviews_5Star)
print(Non_Percentage_5_Star)

61614
48830
0.7925146882202097
